In [1]:
import os
os.environ['MKL_NUM_THREADS']="1"
os.environ['NUMEXPR_NUM_THREADS']="1"
os.environ['OMP_NUM_THREADS']="1" 

In [2]:
import axs
import numpy as np
#import mysql.connector as mariadb
#from astropy.time import Time
import time
import scipy
import pandas as pd

from axs import AxsCatalog, Constants

%matplotlib notebook
import matplotlib.pyplot as plt

#import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import size as spark_size
from pyspark.sql.types import ArrayType, DoubleType
from pyspark.sql.functions import pandas_udf
import pyspark.sql.functions as F

import cesium
from cesium.time_series import TimeSeries
from cesium.featurize import featurize_single_ts, featurize_time_series

from cesium import featurize
import dask

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
pd.set_option('display.max_columns',200)

In [5]:
from dirac import DataBase
db = DataBase(conf={"spark.executor.instances" : "32"})

Status,Pod Name,Message
Running,pyspark-1567589915-stevenstetzler-spark-exec-1,Started container
Running,pyspark-1567589915-stevenstetzler-spark-exec-10,Started container
Running,pyspark-1567589915-stevenstetzler-spark-exec-11,Started container
Running,pyspark-1567589915-stevenstetzler-spark-exec-12,Started container
Running,pyspark-1567589915-stevenstetzler-spark-exec-13,Started container
Running,pyspark-1567589915-stevenstetzler-spark-exec-14,Started container
Running,pyspark-1567589915-stevenstetzler-spark-exec-15,Started container
Running,pyspark-1567589915-stevenstetzler-spark-exec-16,Started container
Running,pyspark-1567589915-stevenstetzler-spark-exec-17,Started container
Running,pyspark-1567589915-stevenstetzler-spark-exec-18,Started container


In [6]:
catalogs = db.get_catalogs()
ztfsample = catalogs.load("ztfsample").exclude_duplicates()
print(ztfsample.count(), len(ztfsample.columns))

10000 11


In [7]:
ztfsample.limit(5).toPandas()

,ra,dec,objectId,jd,dc_mag,dc_sigmag,filter,nobs,__index_level_0__,zone,dup
0,104.750222,-18.384808,ZTF18acuxnow,"[2458462.8323148, 2458464.8473148, 2458464.861...","[12.6912727028392, 12.686325561926463, 12.6684...","[0.008754739519384862, 0.010543184841577045, 0...",2,117.0,6026,4296,0
1,104.750408,-18.384786,ZTF18acuxnoy,"[2458462.8323148, 2458464.8363426, 2458464.837...","[12.719725386451255, 12.699527024088578, 12.70...","[0.004771718142025372, 0.010364937213808078, 0...",2,26.0,6027,4296,0
2,111.194974,-18.399761,ZTF17aaaoiyu,"[2458450.8767823996, 2458464.8427431, 2458464....","[12.42485216233457, 12.260020485191133, 12.234...","[0.010990374237299027, 0.010466264144176316, 0...",2,165.0,6028,4296,0
3,111.194996,-18.399788,ZTF17aaaoiyu,"[2458468.9183796, 2458468.9188426, 2458476.787...","[12.659172478341725, 12.648837263330556, 12.66...","[0.014619226164048475, 0.013554174170469195, 0...",1,13.0,6029,4296,0
4,111.195137,-18.399717,ZTF18acveitr,"[2458464.8372569, 2458464.8418287, 2458464.844...","[12.258117839601704, 12.232431190127217, 12.22...","[0.011003276854977411, 0.010747635222255773, 0...",2,96.0,6030,4296,0


In [8]:
features_General = ["amplitude",
                    "flux_percentile_ratio_mid20",
                    "flux_percentile_ratio_mid35",
                    "flux_percentile_ratio_mid50",
                    "flux_percentile_ratio_mid65",
                    "flux_percentile_ratio_mid80",
                    "max_slope",
                    "maximum",
                    "median",
                    "median_absolute_deviation",
                    "minimum",
                    "percent_amplitude",
                    "percent_beyond_1_std",
                    "percent_close_to_median",
                    "percent_difference_flux_percentile",
                    "period_fast",
                    "qso_log_chi2_qsonu",
                    "qso_log_chi2nuNULL_chi2nu",
                    "skew",
                    "std",
                    "stetson_j",
                    "stetson_k",
                    "weighted_average"]

features_Cadence = ["all_times_nhist_numpeaks",
                    "all_times_nhist_peak1_bin",
                    "all_times_nhist_peak2_bin",
                    "all_times_nhist_peak3_bin",
                    "all_times_nhist_peak4_bin",
                    "all_times_nhist_peak_1_to_2",
                    "all_times_nhist_peak_1_to_3",
                    "all_times_nhist_peak_1_to_4",
                    "all_times_nhist_peak_2_to_3",
                    "all_times_nhist_peak_2_to_4",
                    "all_times_nhist_peak_3_to_4",
                    "all_times_nhist_peak_val",
                    "avg_double_to_single_step",
                    "avg_err",
                    "avgt",
                    "cad_probs_1",
                    "cad_probs_10",
                    "cad_probs_20",
                    "cad_probs_30",
                    "cad_probs_40",
                    "cad_probs_50",
                    "cad_probs_100",
                    "cad_probs_500",
                    "cad_probs_1000",
                    "cad_probs_5000",
                    "cad_probs_10000",
                    "cad_probs_50000",
                    "cad_probs_100000",
                    "cad_probs_500000",
                    "cad_probs_1000000",
                    "cad_probs_5000000",
                    "cad_probs_10000000",
                    "cads_avg",
                    "cads_med",
                    "cads_std",
                    "mean",
                    "med_double_to_single_step",
                    "med_err",
                    "n_epochs",
                    "std_double_to_single_step",
                    "std_err",
                    "total_time"]

features_LS =  ["fold2P_slope_10percentile",
                "fold2P_slope_90percentile",
                "freq1_amplitude1",
                "freq1_amplitude2",
                "freq1_amplitude3",
                "freq1_amplitude4",
                "freq1_freq",
                "freq1_lambda",
                "freq1_rel_phase2",
                "freq1_rel_phase3",
                "freq1_rel_phase4",
                "freq1_signif",
                "freq2_amplitude1",
                "freq2_amplitude2",
                "freq2_amplitude3",
                "freq2_amplitude4",
                "freq2_freq",
                "freq2_rel_phase2",
                "freq2_rel_phase3",
                "freq2_rel_phase4",
                "freq3_amplitude1",
                "freq3_amplitude2",
                "freq3_amplitude3",
                "freq3_amplitude4",
                "freq3_freq",
                "freq3_rel_phase2",
                "freq3_rel_phase3",
                "freq3_rel_phase4",
                "freq_amplitude_ratio_21",
                "freq_amplitude_ratio_31",
                "freq_frequency_ratio_21",
                "freq_frequency_ratio_31",
                "freq_model_max_delta_mags",
                "freq_model_min_delta_mags",
                "freq_model_phi1_phi2",
                "freq_n_alias",
                "freq_signif_ratio_21",
                "freq_signif_ratio_31",
                "freq_varrat",
                "freq_y_offset",
                "linear_trend",
                "medperc90_2p_p",
                "p2p_scatter_2praw",
                "p2p_scatter_over_mad",
                "p2p_scatter_pfold_over_mad",
                "p2p_ssqr_diff_over_var",
                "scatter_res_raw"]

In [9]:
print('total number of time-series features in cesium: \
      {}'.format(len(features_Cadence+features_General+features_LS)))

total number of time-series features in cesium:       112


In [10]:
%%time

lcs = ztfsample.head(100)

CPU times: user 2.12 ms, sys: 3.86 ms, total: 5.98 ms
Wall time: 1.08 s


In [11]:
type(lcs), len(lcs)

(list, 100)

In [12]:
%%time

N = 100
fset_cesium = featurize.featurize_time_series([np.array(lc['jd']) for lc in lcs[:N]],
                                              [np.array(lc['dc_mag']) for lc in lcs[:N]],
                                              [np.array(lc['dc_sigmag']) for lc in lcs[:N]],
                                              features_to_use=features_General+features_LS+features_Cadence,
                                              scheduler=dask.local.get_sync)

fset_cesium

CPU times: user 4.28 s, sys: 25 ms, total: 4.31 s
Wall time: 4.31 s


feature,amplitude,flux_percentile_ratio_mid20,flux_percentile_ratio_mid35,flux_percentile_ratio_mid50,flux_percentile_ratio_mid65,flux_percentile_ratio_mid80,max_slope,maximum,median,median_absolute_deviation,minimum,percent_amplitude,percent_beyond_1_std,percent_close_to_median,percent_difference_flux_percentile,period_fast,qso_log_chi2_qsonu,qso_log_chi2nuNULL_chi2nu,skew,std,stetson_j,stetson_k,weighted_average,fold2P_slope_10percentile,fold2P_slope_90percentile,freq1_amplitude1,freq1_amplitude2,freq1_amplitude3,freq1_amplitude4,freq1_freq,freq1_lambda,freq1_rel_phase2,freq1_rel_phase3,freq1_rel_phase4,freq1_signif,freq2_amplitude1,freq2_amplitude2,freq2_amplitude3,freq2_amplitude4,freq2_freq,freq2_rel_phase2,freq2_rel_phase3,freq2_rel_phase4,freq3_amplitude1,freq3_amplitude2,freq3_amplitude3,freq3_amplitude4,freq3_freq,freq3_rel_phase2,freq3_rel_phase3,freq3_rel_phase4,freq_amplitude_ratio_21,freq_amplitude_ratio_31,freq_frequency_ratio_21,freq_frequency_ratio_31,freq_model_max_delta_mags,freq_model_min_delta_mags,freq_model_phi1_phi2,freq_n_alias,freq_signif_ratio_21,freq_signif_ratio_31,freq_varrat,freq_y_offset,linear_trend,medperc90_2p_p,p2p_scatter_2praw,p2p_scatter_over_mad,p2p_scatter_pfold_over_mad,p2p_ssqr_diff_over_var,scatter_res_raw,all_times_nhist_numpeaks,all_times_nhist_peak1_bin,all_times_nhist_peak2_bin,all_times_nhist_peak3_bin,all_times_nhist_peak4_bin,all_times_nhist_peak_1_to_2,all_times_nhist_peak_1_to_3,all_times_nhist_peak_1_to_4,all_times_nhist_peak_2_to_3,all_times_nhist_peak_2_to_4,all_times_nhist_peak_3_to_4,all_times_nhist_peak_val,avg_double_to_single_step,avg_err,avgt,cad_probs_1,cad_probs_10,cad_probs_20,cad_probs_30,cad_probs_40,cad_probs_50,cad_probs_100,cad_probs_500,cad_probs_1000,cad_probs_5000,cad_probs_10000,cad_probs_50000,cad_probs_100000,cad_probs_500000,cad_probs_1000000,cad_probs_5000000,cad_probs_10000000,cads_avg,cads_med,cads_std,mean,med_double_to_single_step,med_err,n_epochs,std_double_to_single_step,std_err,total_time
channel,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0.297371,0.434013,0.551455,0.703014,0.866573,0.961647,0.176701,15.032929,14.729203,0.241238,14.438187,0.307393,0.333333,0.200000,0.525227,170.537767,3.445861,0.240221,0.238428,0.213018,1.609235,1.149010,14.605970,-1.966606e+00,1.399010e+00,0.229454,0.011755,0.000730,0.001572,7.482105,16.206621,-2.137368,-2.364754,1.801268,2.408870,0.064973,0.005220,0.000995,0.000198,30.048947,1.639437,3.075118,2.572859,0.029846,0.001844,0.000355,0.000329,11.701877,1.866700,2.669225,-0.833654,0.283162,0.130075,4.016109,1.563982,4.467552e-10,5.853759e-09,0.371329,0.0,0.952235,1.000020,0.000017,0.025495,-0.000107,3.272083,0.470277,0.718788,0.358478,1.636557,0.045961,10.0,18.0,20.0,14.0,36.0,1.000000,1.333333,1.333333,1.333333,1.333333,1.000000,0.025795,1.680969e+02,0.026990,2.458417e+06,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.428571,0.714286,0.857143,1.0,1.0,1.0,1.0,1.0,10.548979,4.955278,12.204183,14.688079,-0.451567,0.025299,15.0,7.157597e+02,0.006533,147.685706
1,0.324376,0.243949,0.421284,0.653725,0.854430,0.936556,0.149624,14.136657,13.854548,0.163305,13.487905,0.401706,0.454545,0.090909,0.592909,35.718384,4.080191,-0.014047,-0.250478,0.225278,1.538885,1.069447,13.705739,-8.988076e-06,1.342926e-05,0.266537,0.013404,0.002534,0.000963,32.937974,13.444404,-0.534176,0.470168,-0.121314,1.915818,0.084943,0.010552,0.001339,0.000491,12.489514,1.711057,-2.219618,0.711802,0.020878,0.001070,0.000423,0.000048,6.697617,2.022735,2.165900,-2.271029,0.318691,0.078329,0.379183,0.203340,4.117237e-09,6.375800e-09,0.333602,0.0,1.033806,0.975928,0.000013,-0.000114,0.001304,2.819769,0.862340,1.764981,0.909374,2.275624,0.025202,16.0,16.0,18.0,5.0,7.0,1.333333,2.000000,2.000000,1.500000,1.500000,1.000000,0.035722,-5.992452e+03,0.02